In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.feature_extraction.text import CountVectorizer
import os
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
from tqdm import tqdm
import collections as clt
from textblob import TextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import nltk
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.tokenize import TreebankWordTokenizer
import re
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('movie_reviews')
import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# Seting the directory 
%cd /content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle  

/content/drive/MyDrive/TP_M1_SID/Second_semestre/Compet_kaggle


In [ ]:
df = pd.read_csv('comments_students.csv')[['id', 'ups', 'body']] 
df.head() 

,id,ups,body
0,cqug90j,3.0,No one has a European accent either because i...
1,cqug90k,3.0,That the kid ..reminds me of Kevin. so sad :-(
2,cqug90z,5.0,NSFL
3,cqug91c,1.0,I'm a guy and I had no idea this was a thing g...
4,cqug91e,101.0,"Mid twenties male rocking skinny jeans/pants, ..."


In [ ]:
df.shape

(4234970, 3)

## 1. Sentiment analysis
### Before cleaning the text, let's do some sentiment analysis

In [ ]:
df['body'].isnull().sum()
print("have 61 rows which don't have any comment")
print('----------------------------------------------------------')
# Let's replace nan values by white space 
df['body'].fillna('', inplace = True)
# Then we have 
print(df['body'].isnull().sum())

have 61 rows which don't have any comment
----------------------------------------------------------
0


##### I had the intuition that the score of the comment is determined by it's text represented by the variable "body". More specifically by the feeling it gave off: according to the subjectivity (the expression of the author's opinion, 0 for a very little subjective comment and 1 for an extremely subjective comment) and polarity (the level of emotion, -1 for a negative comment and +1 for a very positive comment). We used the library called textBlob. 

In [ ]:
## These two function help us to get the polarity and subjectivity of a sentence 
def textBlobPolarity(sent):
  polarity = TextBlob(sent).sentiment.polarity
  return polarity

def textBlobSubjectivity(sent):
  subjectivity = TextBlob(sent).sentiment.subjectivity
  return subjectivity


In [ ]:
def sent_analysis(col, nb_chunk, function_name, df):
  """
  description: enables to analyse sentiment in text according to subjectivity or polarity
  in: 
    col: name of the column we add to the dataframe
    nb_chunk: number of chunks used
    function_name: can be textBlobPolarity or textBlobSubjectivity
    df: our dataframe which contains our data
  out: the dataframe where we add the sentiment analysis columns    
  """
  df[str(col)] = df['body']

  df_chunk =  np.array_split(df, nb_chunk)
  chunk_list = []
  for i, chunk in tqdm(enumerate(df_chunk)):
    chunk_filter = chunk
    chunk_filter[str(col)] = chunk['body'].apply(function_name)
    chunk_list.append(chunk_filter) #Normally chunk_filter 
      
  # concat the list into dataframe 
  return pd.concat(chunk_list)  


In [ ]:
def convertion(x):
  if x<0:
    return -1
  elif x> 0:
    return 1
  return 0

In [ ]:
# Applying the function sent_analysis to get the polarity of each comment 

from tqdm import tqdm 
df = sent_analysis('polarity', 1000, textBlobPolarity, df)

1000it [23:11,  1.39s/it]


In [ ]:
# Applying the function sent_analysis to get the polarity of each comment 
from tqdm import tqdm
df = sent_analysis('subjectivity', 1000, textBlobSubjectivity, df)

1000it [23:12,  1.39s/it]


In [ ]:
%%time
# To avoid a problem of disrimatory, we try and convert polarity values. Thus we get only three possible vlaues 0 = neutral, -1 = negative, and 1 = positive 
df['polarity'] = df['polarity'].apply(convertion)

CPU times: user 2.07 s, sys: 60.1 ms, total: 2.13 s
Wall time: 2.13 s


In [ ]:
df

,id,ups,body,polarity,subjectivity
0,cqug90j,3.0,No one has a European accent either because i...,0,0.000000
1,cqug90k,3.0,That the kid ..reminds me of Kevin. so sad :-(,-1,1.000000
2,cqug90z,5.0,NSFL,0,0.000000
3,cqug91c,1.0,I'm a guy and I had no idea this was a thing g...,0,0.000000
4,cqug91e,101.0,"Mid twenties male rocking skinny jeans/pants, ...",1,0.427083
...,...,...,...,...,...
4234965,crrbelu,NaN,Does the sun set in the west/rise in the east?...,0,0.000000
4234966,crrbelv,NaN,Coffee.,0,0.000000
4234967,crrbemp,NaN,"people who cannot make up their mind, my bulls...",-1,0.400000
4234968,crrbenh,NaN,Give them to Irish people in exchange for doin...,0,0.000000


## Saving a copy

In [ ]:
df.to_csv('sentiment_analysis.csv', index=False) 

In [ ]:
df = pd.read_csv("sentiment_analysis.csv", lineterminator='\n')
df

,id,ups,body,polarity,subjectivity
0,cqug90j,3.0,No one has a European accent either because i...,0,0.000000
1,cqug90k,3.0,That the kid ..reminds me of Kevin. so sad :-(,-1,1.000000
2,cqug90z,5.0,NSFL,0,0.000000
3,cqug91c,1.0,I'm a guy and I had no idea this was a thing g...,0,0.000000
4,cqug91e,101.0,"Mid twenties male rocking skinny jeans/pants, ...",1,0.427083
...,...,...,...,...,...
4234965,crrbelu,NaN,Does the sun set in the west/rise in the east?...,0,0.000000
4234966,crrbelv,NaN,Coffee.,0,0.000000
4234967,crrbemp,NaN,"people who cannot make up their mind, my bulls...",-1,0.400000
4234968,crrbenh,NaN,Give them to Irish people in exchange for doin...,0,0.000000


# Text Cleaning 
I clean data from variable 'body' in order to improve the the performance of our models. For instance converting to lower case ( KIDS will be equal to kids). 


In [ ]:
# Loading english stop words
stop = stopwords.words('english')
# Adding some characters to the loaded punctuation
punctuations = punctuation+"’”“‘…„—᾿‐–‑′•›‹⁄―‚→（）『』》《。↓↵'͞ʻʿ'"+'″￼'

## Function that clean the body content
def text_cleaning(sentence):
  """ Documentation
    text :input parameter, must be a string 
    Returns the vocabulary without accent
    Out : string
    """

  sentence = str(sentence) # Converting to str if we get a non str type
  # email format
  email =re.compile('[a-zA-Z0-9.-]+@[a-zA-Z._/-]+.[a-z]+')
  # caracteres
  caracters=re.compile('\W+')
  # number 
  number =re.compile('[0-9]+' )

  # Convert to lowercase
  sentence = sentence.lower() 
  # Removing stopwords
  sentence = ' '.join([word for word in sentence.split() if word not in (stop)])  

  # Tranforming the sentence into a list of token 
  liste_tokens = TreebankWordTokenizer().tokenize(sentence)

  # Removing sepecial caratere from the token: eg ""..reminds" becomes "reminds"
  for i in range(len(liste_tokens)):
    liste_tokens[i] = liste_tokens[i].strip(punctuations)

  
  
  # Remove number
  sentence = number.sub(" ", sentence)
  # 
  sentence = caracters.sub(" ", sentence)
  # handling email format
  sentence = email.sub(" ", sentence)

  # Remove words which len are <2
  sentence = ' '.join([w for w in liste_tokens if len(w) > 1])
  
  return sentence

In [ ]:
df_chunk =  np.array_split(df, 100)     

In [ ]:
%%time 

chunk_list = [] 

for i, chunk in tqdm(enumerate(df_chunk)):
    chunk_filter = chunk
    chunk_filter['body'] = chunk['body'].apply(text_cleaning)
    
    # Once the data filtering is done, append the chunk to list 
    chunk_list.append(chunk_filter) #Normally chunk_filter 
    
# concat the list into dataframe 
df = pd.concat(chunk_list)  

100it [13:21,  8.01s/it]


CPU times: user 13min 12s, sys: 3.42 s, total: 13min 16s
Wall time: 13min 21s


### Lemmatization
#### For the sake of improving our model, let's do some lemmatization for instance accents becomes accent. 

In [ ]:
lemmer=WordNetLemmatizer()

def lemmatization(word):
  """
  description: enable to consider for each term only its root meaning.
  """
  for pos in ['a', 's', 'r', 'n', 'v']:
    word=lemmer.lemmatize(word,pos)
  return word

In [ ]:
%%time 
# Applying the lemmmatization function on each comment 

df["body"] = df["body"].apply(lambda x: ' '.join([lemmatization(word) for word in x.split() ]))

CPU times: user 16min 31s, sys: 6.74 s, total: 16min 38s
Wall time: 16min 43s


## Saving a copy

In [ ]:
df.to_csv('body_clean_lem.csv', index = False)

,id,ups,body,polarity,subjectivity
0,cqug90j,3.0,one european accent either exist accent europe...,0.000000,0.000000
1,cqug90k,3.0,kid remind kevin sad,-0.625000,1.000000
2,cqug90z,5.0,nsfl,0.000000,0.000000
3,cqug91c,1.0,guy idea thing guy do,0.000000,0.000000
4,cqug91e,101.0,mid twenty male rock skinny jeans/pants style ...,0.189062,0.427083
...,...,...,...,...,...
4234965,crrbelu,NaN,sun set west/rise east yes,0.000000,0.000000
4234966,crrbelv,NaN,coffee,0.000000,0.000000
4234967,crrbemp,NaN,people can not make mind bullshit ex slow wifi,-0.300000,0.400000
4234968,crrbenh,NaN,give irish people exchange bid,0.000000,0.000000


## Re-loading data 

In [ ]:
df_clean = pd.read_csv('body_clean_lem.csv')
df_clean 

,id,ups,body
0,cqug90j,3.0,one european accent either exist accent europe...
1,cqug90k,3.0,kid remind kevin sad
2,cqug90z,5.0,nsfl
3,cqug91c,1.0,guy idea thing guy do
4,cqug91e,101.0,mid twenty male rock skinny jeans/pants style ...
...,...,...,...
4234965,crrbelu,NaN,sun set west/rise east yes
4234966,crrbelv,NaN,coffee
4234967,crrbemp,NaN,people can not make mind bullshit ex slow wifi
4234968,crrbenh,NaN,give irish people exchange bid


In [ ]:
df_clean.shape

(4234970, 3)

In [ ]:
df_clean['body'].isnull().sum()

29927

In [ ]:
x = df_clean['body'].isnull().sum()
print(f"have {x} rows which don't have any comment")
print('----------------------------------------------------------')
# Let's replace nan values by white space 
df_clean['body'].fillna('', inplace = True)
print('Then we have ')
print(df_clean['body'].isnull().sum())

have 29927 rows which don't have any comment
----------------------------------------------------------
Then we have 
0


# Calculating the tf_idf values 
I calculated tf-idf values but it was not used in the prediction because it deteriorated results

In [ ]:
%%time
vectorizer = TfidfVectorizer(max_df=0.95, min_df= 5000, max_features= 200000, stop_words='english')

tfidf = vectorizer.fit_transform(df_clean['body']) 

CPU times: user 1min 6s, sys: 1.02 s, total: 1min 7s
Wall time: 1min 7s


In [ ]:
tfidf.shape

(4234970, 1394)

In [ ]:
%%time
df_clean['tf_idf'] = tfidf
df_clean = df_clean[['id', 'ups', 'body']]

CPU times: user 651 ms, sys: 6.69 ms, total: 658 ms
Wall time: 663 ms


# Topic extraction 

I tried to extract topics from all the text of the comments. A comment can have several topics. As a consequence, we had the intuition that a comment which belongs to a lot of topics has more chance to have a great score and vice-versa.

In [ ]:
# before we apply our LDA let'us create a vocabulary of all our data 

count_vect = CountVectorizer(min_df=5000,max_df=200000, stop_words='english')
X = count_vect.fit_transform(df_clean['body'])

# Fetching words from our vocabulary
colonnes=count_vect.get_feature_names()
X.shape

(4234970, 1384)

In [ ]:
%%time
# Creating a LDA, let's specify the number of categorie or topic into 100
nb_topics=100
LDA = LatentDirichletAllocation(n_components=nb_topics, learning_method= 'online', learning_offset=50.,random_state=42)

# Fiiting on data 
LDA.fit(X)

CPU times: user 3h 41s, sys: 1h 8s, total: 4h 49s
Wall time: 2h 56min 49s


###### Saving our LDA model

In [ ]:
filename = 'lda_model.sav'
pickle.dump(LDA, open(filename, 'wb'))


#### Re-loading the model

In [ ]:
filename = 'lda_model.sav'

lda_model = pickle.load(open(filename, 'rb'))

In [ ]:
#Evaluation, getting the 15 most frequent for each topic 
n_terms=20
dict_topics=clt.defaultdict(list)
L=lda_model.components_

for i, r in enumerate(L):
    for i in r.argsort()[:-n_terms - 1:-1]:
          dict_topics[i+1].append(colonnes[i])  

In [ ]:
def sent_topic_weight(doc):
    """
    descritpion: A function that assigns each document a weight, (the number of topics the document talks about it )/100.
    """
    doc=str(doc).split()
    n=0
    topic_compter=[]
    for elt in doc:
        for i in range(1,101):
            if i not in topic_compter:
                if elt in dict_topics[i]:
                    n+=1
                    topic_compter.append(i)
    return n/100 

In [ ]:
%%time 

## We apply the function sent_topic_weight to each row of our dataframe, so we assign each comment a weight 

df_clean['weight_topic']=df_clean['body'].apply(func=sent_topic_weight)
df_clean.head()

CPU times: user 30min 50s, sys: 3.52 s, total: 30min 53s
Wall time: 31min 13s


In [ ]:
df_clean 

,id,ups,body,weight_topic
0,cqug90j,3.0,one european accent either exist accent europe...,0.00
1,cqug90k,3.0,kid remind kevin sad,0.00
2,cqug90z,5.0,nsfl,0.00
3,cqug91c,1.0,guy idea thing guy do,0.00
4,cqug91e,101.0,mid twenty male rock skinny jeans/pants style ...,0.02
...,...,...,...,...
4234965,crrbelu,NaN,sun set west/rise east yes,0.00
4234966,crrbelv,NaN,coffee,0.00
4234967,crrbemp,NaN,people can not make mind bullshit ex slow wifi,0.00
4234968,crrbenh,NaN,give irish people exchange bid,0.00


In [ ]:
# Scaling data 
from sklearn.preprocessing import StandardScaler

df_copy = df_clean[['id', 'weight_topic']]
scaler = StandardScaler()
df_copy.iloc[:,1:]=scaler.fit_transform(df_copy.iloc[:,1:])
df_copy


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,id,weight_topic
0,cqug90j,-0.400798
1,cqug90k,-0.400798
2,cqug90z,-0.400798
3,cqug91c,-0.400798
4,cqug91e,2.477223
...,...,...
4234965,crrbelu,-0.400798
4234966,crrbelv,-0.400798
4234967,crrbemp,-0.400798
4234968,crrbenh,-0.400798


### Saving a copy

In [ ]:
df_copy.to_csv("topic.csv", index = False)